In [1]:
# So we can run this from python2 or python3.
# But you should be using python3!!!
from __future__ import print_function

# Submit

Tools run on the hubs are limited because they run in a container on an execution host.  Many simulations require large amounts or memory and/or CPU time.  They require access to external HPC or grid resources.  

**submit** takes a user command and executes it remotely. The objective is to allow the user to issue a command in the same manner as a locally executed command. Multiple submission mechanisms are available for run dissemination. A set of steps are executed for each run submission:

* Destination site is selected
* A wrapper script is generated for remote execution.

* If needed a batch system description file is generated.
* Input files for a run are gathered and transferred to the remote site. Transferred files include the wrapper and batch description scripts.
* The wrapper script is executed remotely.
* Progress of the remote run is monitored until completion.
* Output files from the run are returned to the dissemination point.


```bash
> submit --help
Usage: submit [options]

Options:
  -h, --help            Report command usage. Optionally request listing of
                        managers, tools, venues, or examples.
  -l, --local           Execute command locally
  --status              Report status for runs executing remotely.
  -k, --kill            Kill runs executing remotely.
  --venueStatus         Report venue status.
  -v, --venue           Remote job destination
  -i, --inputfile       Input file
  -p, --parameters      Parameter sweep variables. See examples.
  -d, --data            Parametric variable data - csv format
  -s SEPARATOR, --separator=SEPARATOR
                        Parameter sweep variable list separator
  -n NCPUS, --nCpus=NCPUS
                        Number of processors for MPI execution
  -N PPN, --ppn=PPN     Number of processors/node for MPI execution
  -w WALLTIME, --wallTime=WALLTIME
                        Estimated walltime hh:mm:ss or minutes
  -e, --env             Variable=value
  --runName=RUNNAME     Name used for directories and files created during the
                        run. Restricted to alphanumeric characters
  -m, --manager         Multiprocessor job manager
  -r NREDUNDANT, --redundancy=NREDUNDANT
                        Number of indentical simulations to execute in
                        parallel
  -M, --metrics         Report resource usage on exit
  --detach              Detach client after launching run
  --attach=ATTACHID     Attach to previously detached started server
  -W, --wait            Wait for reduced job load before submission
  -Q, --quota           Enforce local user quota on remote execution host
  -q, --noquota         Do not enforce local user quota on remote execution
                        host
  --tailStdout          Periodically report tail of stdout file.
  --tailStderr          Periodically report tail of stderr file.
  --tail                Periodically report tail of application file.
  --progress            Show progress method. Choices are auto, curses,
                        submit, text, pegasus, or silent.
```


# Learning by Doing

This notebook will demonstrate how to use submit.  First it will show command line usage that will not require Python.  Then it will show some how to use a python convenience function to run submit.


## Venues

Submit can submit to different hosts or clusters. You can get a list for your hub by asking submit:


In [2]:
!submit --help venues


Currently available VENUES are:
   OSGFactory
   RedCloud
   compute@comet_nanobio
   conte
   datalimited@rice
   ncn-hub@conte
   ncn-hub@rice
   ncn-hub_L@conte
   ncn-hub_L@rice
   ncn-hub_M@conte
   ncn-hub_M@rice
   ncn-hub_S@conte
   ncn-hub_S@rice
   ncn-hub_XL@conte
   ncn-hub_XL@rice
   ncn@conte
   ncn@rice
   ncn_L@conte
   ncn_L@rice
   ncn_M@conte
   ncn_M@rice
   ncn_S@conte
   ncn_S@rice
   ncn_XL@conte
   ncn_XL@rice
   partner@halsteadgpu
   punch
   rcac-ncn
   rcac-ncn-hub
   rcac-standby
   rcac_L
   rcac_M
   rcac_S
   rcac_XL
   rice
   shared@comet_nanobio
   standby@conte
   standby@rice
   standby_L@conte
   standby_L@rice
   standby_M@conte
   standby_M@rice
   standby_S@conte
   standby_S@rice
   standby_XL@conte
   standby_XL@rice
   submithost


<div class="alert alert-warning">
How do we know what the default venue is?
How should we choose the venue to use?
</div>

### What commands can I submit?

#### Submitting locally with --local

Any executable can be used when submitting locally.  Local submit runs in the current
execution host in the current container, so there are few reasons to use it.  However, it
does run quickly, so it is useful for quick tests such as the ones we run in this notebook.

#### Submitting to the grid

Only executables staged in /apps
can be submitted to the grid.  

In [3]:
!submit --local echo hi

hi


## Submitting with Parameters

Parameter examples:

submit -p @@cap=10pf,100pf,1uf sim.exe @:indeck

    Submit 3 jobs. The @:indeck means "use the file indeck as a template
    file." Substitute the values 10pf, 100pf, and 1uf in place of @@cap within the
    file. Send off one job for each of the values and bring back the results.

submit -p @@vth=0:0.2:5 -p @@cap=10pf,100pf,1uf sim.exe @:indeck

    Submit 78 jobs. The parameter @@vth goes from 0 to 5 in steps of 0.2,
    so there are 26 values for @@vth. For each of those values, the parameter
    @@cap changes from 10pf to 100pf to 1uf. 26 x 3 = 78 jobs total. Again
    @:indeck is treated as a template, and the values are substituted in place of
    @@vth and @@cap in that file.

submit -p params sim.exe @:indeck

    In this case, parameter definitions are taken from the file named
    params instead of the command line. The file might have the following
    contents:

        # paramters for my job submission
        parameter @@vth=0:0.2:5
        parameter @@cap = 10pf,100pf,1uf

submit -p "params;@@num=1-10;@@color=blue" job.sh @:job.data

    For someone who loves syntax and complexity... The semicolon separates
    the parameters value into three parts. The first says to load parameters from
    a file params. The next part says add an additional parameter @@num that goes
    from 1 to 10. The last part says add an additional parameter @@color with a
    single value blue. The parameters @@num and @@color cannot override anything
    defined within params; they must be new parameter names.

submit -d input.csv sim.exe @:indeck

    Takes parameters from the data file input.csv, which must be in comma-
    separated value format. The first line of this file may contain a series of
    @@param names for each of the columns. Whitespace is significant for all
    values entered in the csv file. If it doesn't, then the columns are assumed to
    be called @@1, @@2, @@3, etc. Each of the remaining lines represents a set of
    parameter values for one job; if there are 100 such lines, there will be 100
    jobs. For example, the file input.csv might look like this:

        @@vth,@@cap
        1.1,1pf
        2.2,1pf
        1.1,10pf
        2.2,10pf

    Parameters are substituted as before into template files such as
    @:indeck.

submit -d input.csv -p "@@doping=1e15-1e17 in 30 log" sim.exe @:infile

    Takes parameters from the data file input.csv, but also adds another
    parameter @@doping which goes from 1e15 to 1e17 in 30 points on a log scale.
    For each of these points, all values in the data file will be executed. If the
    data file specifies 50 jobs, then this command would run 30 x 50 = 1500 jobs.

submit -d input.csv -i @:extra/data.txt sim.exe @:indeck

    In addition to the template indeck file, send along another file
    extra/data.txt with each job, and treat it as a template too.

submit -s / -p @@address=23 Main St.,Hometown,Indiana/42 Broadway,Hometown,Indiana -s , -p @@color=red,green,blue job.sh @:job.data

    Change the separator to slash when defining the addresses, then change
    back to comma for the @@color parameter and any remaining arguments. We
    shouldn't have to change the separator often, but it might come in handy if
    the value strings themselves have commas.

submit -p @@num=1:1000 sim.exe input@@num

    Submit jobs 1,2,3,...,1000. Parameter names such as @@num are
    recognized not only in template files, but also for arguments on the command
    line. In this case, the numbers 1,2,3,...,1000 are substituted into the file
    name, so the various jobs take their input from "input1", "input2", ...,
    "input1000".

submit -p @@file=glob:indeck* sim.exe @@file

    Look for files matching indeck* and use the list of names as the
    parameter @@file. Those values could be substituted into other template files,
    or used on the command line as in this example. Suppose the directory contains
    files indeck1, indeck10, and indeck2.  The glob option will order the files in
    a natural order: indeck1, indeck2, indeck10.  This example would launch three
    jobs using each of those files as input for the job.

submit -p @@file=globnat:indeck* sim.exe @@file

    This option has been deprecated.  The functionality is now available with the glob option.

Let's try the echo program with a list of different input parameters.  To see progress, we need to use "text" or "submit".  Other options won't work well with Jupyter.


In [4]:
# need this because this example is running in a read-only directory
import os
try:
    os.mkdir('/tmp/submit')
except:
    pass
os.chdir('/tmp/submit')

In [5]:
!submit --local --progress text -p @@name=hub1,hub2,hub3 echo @@name

Simulations complete. Results are stored in directory /tmp/submit/06614631


Notice how the outputs were written to subdirectories in a directory that was created for us.
If we want to access those results more conveniently, it would be best to tell submit what name to use for the directory.
We can do this by using the **--runName** parameter.
<div class="alert alert-info">
If the directory exists, submit will complain and exit immediately.

</div>

In [6]:
!rm -rf echotest
!submit --local --runName=echotest --progress submit -s, -p @@name=hub1,hub2,hub3 echo @@name

=SUBMIT-PROGRESS=> aborted=0 finished=0 failed=0 executing=0 waiting=0 setting_up=0 setup=3 %done=0.00 timestamp=1525204621.7
=SUBMIT-PROGRESS=> aborted=0 finished=0 failed=0 executing=1 waiting=0 setting_up=0 setup=2 %done=0.00 timestamp=1525204626.0
=SUBMIT-PROGRESS=> aborted=0 finished=1 failed=0 executing=0 waiting=0 setting_up=0 setup=2 %done=33.33 timestamp=1525204626.0
=SUBMIT-PROGRESS=> aborted=0 finished=1 failed=0 executing=1 waiting=0 setting_up=0 setup=1 %done=33.33 timestamp=1525204626.2
=SUBMIT-PROGRESS=> aborted=0 finished=2 failed=0 executing=0 waiting=0 setting_up=0 setup=1 %done=66.67 timestamp=1525204626.2
=SUBMIT-PROGRESS=> aborted=0 finished=2 failed=0 executing=1 waiting=0 setting_up=0 setup=0 %done=66.67 timestamp=1525204626.4
=SUBMIT-PROGRESS=> aborted=0 finished=3 failed=0 executing=0 waiting=0 setting_up=0 setup=0 %done=100.00 timestamp=1525204626.4


Once submit finishes, we can see the standard output for each job in the "echotext" subdirectory.  Standard output  will be in echotest_{jobnum}.stdout.

In [7]:
!ls -lR echotest

echotest:
total 20
drwxr-x--- 2 mmh public 4096 May  1 15:57 01
drwxr-x--- 2 mmh public 4096 May  1 15:57 02
drwxr-x--- 2 mmh public 4096 May  1 15:57 03
drwxr-x--- 3 mmh public 4096 May  1 15:57 mmh
-rw-r----- 1 mmh public  197 May  1 15:57 parameterCombinations.csv

echotest/01:
total 4
-rw-rw-r-- 1 mmh public 5 May  1 15:57 echotest_01.stdout

echotest/02:
total 4
-rw-rw-r-- 1 mmh public 5 May  1 15:57 echotest_02.stdout

echotest/03:
total 4
-rw-rw-r-- 1 mmh public 5 May  1 15:57 echotest_03.stdout

echotest/mmh:
total 4
drwxr-x--- 3 mmh public 4096 May  1 15:57 pegasus

echotest/mmh/pegasus:
total 4
drwxr-x--- 2 mmh public 4096 May  1 15:57 06614632

echotest/mmh/pegasus/06614632:
total 0


In [8]:
for i in range(1,4):
    with open("echotest/%02d/echotest_%02d.stdout" % (i, i), 'r') as f:
        print("Run %02d: %s" % (i, f.read()))

Run 01: hub1

Run 02: hub2

Run 03: hub3



In [9]:
!cat echotest/parameterCombinations.csv

# Using runCommand

Using the exclamation point(!) to run shell commands from Jupyter is fine for many cases.  But often you want to call submit from a python function or check the return value.  hublib provides some very flexible python functions to do this  https://hubzero.github.io/hublib/cmd.html

In [10]:
from hublib.cmd import runCommand
!rm -rf runtest
res, stdout, stderr = runCommand("submit --runName=runtest --progress submit -p @@Vin=1,2,3,4,5 /apps/pegtut/current/examples/capacitor_voltage/sim1.py  --Vin @@Vin")

=SUBMIT-PROGRESS=> aborted=0 finished=0 failed=0 executing=0 waiting=5 setting_up=0 setup=0 %done=0.00 timestamp=1525204693.1
=SUBMIT-PROGRESS=> aborted=0 finished=5 failed=0 executing=0 waiting=0 setting_up=0 setup=0 %done=100.00 timestamp=1525204791.0
Simulations complete. Results are stored in directory /home/nanohub/mmh/data/sessions/1301182/run_NyUMtd/runtest


Return code for normal completion is zero.

In [11]:
res

0

For the last example, we used a test program installed on nanohub so we didn't have to execute locally.  The program, "sim1.py" takes a single argument and outputs values to a file, "out.log".  It does not write to stdout.  You can see all the output in the subdirectories under "runtest"

In [12]:
!ls -lR runtest

runtest:
total 48
drwxr-x--- 2 mmh public 4096 May  1 15:59 01
drwxr-x--- 2 mmh public 4096 May  1 15:59 02
drwxr-x--- 2 mmh public 4096 May  1 15:59 03
drwxr-x--- 2 mmh public 4096 May  1 15:59 04
drwxr-x--- 2 mmh public 4096 May  1 15:59 05
-rw-r----- 1 mmh public  271 May  1 15:59 parameterCombinations.csv
-rw-r----- 1 mmh public 1104 May  1 15:59 pegasus.analysis
-rw-r----- 1 mmh public 2214 May  1 15:59 pegasusjobstats.csv
-rw-r----- 1 mmh public 2504 May  1 15:59 pegasusjobstats.txt
-rw-r----- 1 mmh public  148 May  1 15:59 pegasusstatus.txt
-rw-r----- 1 mmh public 1953 May  1 15:59 pegasussummary-time.csv
-rw-r----- 1 mmh public 1260 May  1 15:59 pegasussummary.csv

runtest/01:
total 0

runtest/02:
total 0

runtest/03:
total 0

runtest/04:
total 0

runtest/05:
total 0


# Using SubmitCommand class

Furthur Python integration is achieved by using the hubzero.submit.SubmitCommand class.  Methods are provided for specifying any and all **submit** arguments.  For each **submit** command argument there are typically two method types - set and reset. The standard python **help** builtin can be used to show methods and associated arguments.  The code shown here can also be used Rappture Python wrapper scripts.

In [13]:
from hubzero.submit.SubmitCommand import SubmitCommand

submitCommand = SubmitCommand()
help (submitCommand)

Help on SubmitCommand in module hubzero.submit.SubmitCommand object:

class SubmitCommand(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, configurationDirectory='/etc/submit', hubLogPath='/tmp/submit/.submit.log')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  addParameters(self, parameters, separator=None)
 |      Supply additional set of parameters for submit run.
 |      parameters can be specified as a single string or
 |      of list of strings.
 |  
 |  loadSubmitCommand(self, submitCommandJSONFile)
 |      Load JSON file containing submit command settings.
 |  
 |  resetAttachId(self)
 |      Do not reattach to a previously detached submit run.
 |  
 |  resetCommand(self)
 |      Remove previous command and command argument settings.
 |  
 |  resetCommandArguments(self)
 |      Remove previous command argument settings.
 |  
 |  resetDataFile(self)
 |      Remove parameter datafile for submit run.
 |  
 |  resetDebug(self)
 |  

Help about the underlying **submit** command can be easily reported.

In [14]:
submitCommand = SubmitCommand()
result = submitCommand.submit(['--help'])

Usage: submit [options]

Options:
  -h, --help            Report command usage. Optionally request listing of
                        managers, tools, venues, or examples.
  -l, --local           Execute command locally
  --status              Report status for runs executing remotely.
  -k, --kill            Kill runs executing remotely.
  --venueStatus         Report venue status.
  -v, --venue           Remote job destination
  -i, --inputfile       Input file
  -p, --parameters      Parameter sweep variables. See examples.
  -d, --data            Parametric variable data - csv format
  -s SEPARATOR, --separator=SEPARATOR
                        Parameter sweep variable list separator
  -n NCPUS, --nCpus=NCPUS
                        Number of processors for MPI execution
  -N PPN, --ppn=PPN     Number of processors/node for MPI execution
  -w WALLTIME, --wallTime=WALLTIME
                        Estimated walltime hh:mm:ss or minutes
  -e, --env             Variable=value
  --runNa

A listing of available venues can be listed

In [15]:
submitCommand = SubmitCommand()
submitCommand.setHelp(detail='venues')
result = submitCommand.submit()


Currently available VENUES are:
   OSGFactory
   RedCloud
   compute@comet_nanobio
   conte
   datalimited@rice
   ncn-hub@conte
   ncn-hub@rice
   ncn-hub_L@conte
   ncn-hub_L@rice
   ncn-hub_M@conte
   ncn-hub_M@rice
   ncn-hub_S@conte
   ncn-hub_S@rice
   ncn-hub_XL@conte
   ncn-hub_XL@rice
   ncn@conte
   ncn@rice
   ncn_L@conte
   ncn_L@rice
   ncn_M@conte
   ncn_M@rice
   ncn_S@conte
   ncn_S@rice
   ncn_XL@conte
   ncn_XL@rice
   partner@halsteadgpu
   punch
   rcac-ncn
   rcac-ncn-hub
   rcac-standby
   rcac_L
   rcac_M
   rcac_S
   rcac_XL
   rice
   shared@comet_nanobio
   standby@conte
   standby@rice
   standby_L@conte
   standby_L@rice
   standby_M@conte
   standby_M@rice
   standby_S@conte
   standby_S@rice
   standby_XL@conte
   standby_XL@rice
   submithost


A listing of available staged tools can be listed

In [16]:
submitCommand = SubmitCommand()
submitCommand.setHelp(detail='tools')
result = submitCommand.submit()


Currently available TOOLs are:
   1dchainmd_r15
   1dfdmht_r28
   CAC-runapp
   abase_r9
   abinit-6.12.3
   abinit-7.10.4
   acmenpds-r49
   adept_r32
   adeptnpt_r151
   advte_r15
   ahpull_r27
   amobt_r51
   angel_r50
   annihilate_r28
   arbd-r15
   arbd-r3
   archimedes_r71
   asts_r8
   berkeleygw-1178_sigma
   berkeleygw-1178_xi0
   biconvex-r15
   binbrush_r15
   biolab_r64
   biomoca_r29
   biophotonicsim-r7
   bmcsuite_r44
   bowtie_r165
   bsclab_r32
   btesolver_r41
   bulkmc_r39
   bulkmobility_r13
   cadnanovis_r41
   carrierconc_r12
   ccamt_r12
   cdlev_r10
   circuitelements_r4
   cndo_r38
   cnia_r20
   cnia_r24
   cntbands-ext_r41
   cntbte_r25
   cntmob_r44
   cntnemscant_r7
   cntnemsff_r7
   cntphonons_r25
   cntrelay_r25
   complam_r35
   cond3d_r14
   contactmaps_r46
   coulombsim_r20
   crystal_viewer_r398
   cvgraph_r26
   dbrlaser_r5
   ddaconvert_r69-pip
   ddscat-7.3.0-intel-14_mpi
   ddscat-7.3.0-intel-14_openmp
   ddscat-7.3.0-intel-14_serial
   ddsolar

   espresso-6.2.1_plan_avg
   espresso-6.2.1_plotband
   espresso-6.2.1_plotproj
   espresso-6.2.1_plotrho
   espresso-6.2.1_pmw
   espresso-6.2.1_pp
   espresso-6.2.1_projwfc
   espresso-6.2.1_pw
   espresso-6.2.1_pw2bgw
   espresso-6.2.1_pw2gw
   espresso-6.2.1_pw2wannier90
   espresso-6.2.1_pw4gww
   espresso-6.2.1_pw_export
   espresso-6.2.1_pwcond
   espresso-6.2.1_pwi2xsf
   espresso-6.2.1_q2qstar
   espresso-6.2.1_q2r
   espresso-6.2.1_q2trans
   espresso-6.2.1_q2trans_fd
   espresso-6.2.1_spectra_correction
   espresso-6.2.1_sumpdos
   espresso-6.2.1_turbo_davidson
   espresso-6.2.1_turbo_eels
   espresso-6.2.1_turbo_lanczos
   espresso-6.2.1_turbo_spectrum
   espresso-6.2.1_wannier_ham
   espresso-6.2.1_wannier_plot
   espresso-6.2.1_wfck2r
   espresso-6.2.1_wfdd
   espresso-6.2.1_xspectra
   exdynamics_r93
   fbsdcme_r33
   fdical_r24
   fdiscovery_r17
   fermi_r83
   fettoy_r41
   ffettool_r18
   freeedge_r46
   gamess-20130501
   gamess-20170420
   geneticalgo_r12
   gfetto

For demonstration purposes we can use a Python application that is part of the Pegasus Tutorial tool. Because it is installed in the /apps directory it is available for submisison to most venues.

In [17]:
applicationCode = '/apps/pegtut/current/examples/capacitor_voltage/sim1.py'

The SubmitCommand class has a method that excepts command arguments as a simple list.

In [18]:
submitCommand = SubmitCommand()
result = submitCommand.submit(['-w','5',applicationCode,'--C','0.001','--Vin','3'])
print(result)

Run 6614639 registered 1 job instance. Tue May  1 16:00:07 2018
Run 6614639 instance 1 released for submission. Tue May  1 16:00:45 2018
(945675.0) Job Submitted at OSGFactory Tue May  1 16:00:51 2018
(945675.0) Job Done at OSGFactory Tue May  1 16:02:16 2018
{'jobId': 6614639, 'runName': '06614639', 'exitCode': 0}


You should notice that the single run resulted in submission of jobs to three different sites.  When no venue is specified and the application is not staged at any remote venues redundant jobs are submited.  When one job completes sucessfully the other jobs are terminated.
The result dictionary has three members.  The *jobId* is the unique identifier given by **submit** to every run.  *runName* is the name of the run and by default it matches the *jobId*.  *runName* can be optionally set with **submit** arguments.  Standard output and standard error files generated by remote application execution are returned as *runName*.stdout and *runName*.stderr.  The standard error file will not be returned if it is empty. 

In [19]:
runStdout = result['runName'] + '.stdout'
!cat $runStdout

./sim1.py --C 0.001 --Vin 3


In [20]:
runStderr = result['runName'] + '.stderr'
!cat $runStderr

cat: 06614639.stderr: No such file or directory


Using the set methods and specifying a particular venue

In [21]:
submitCommand = SubmitCommand()
submitCommand.setWallTime(5)
submitCommand.setVenue('OSGFactory')
submitCommand.setCommand(applicationCode)
submitCommand.setCommandArguments(['--C','0.001','--Vin','3'])
submitCommand.show()
result = submitCommand.submit()

submit --venue OSGFactory --wallTime 5
       /apps/pegtut/current/examples/capacitor_voltage/sim1.py --C 0.001 --Vin 3
Run 6614643 registered 1 job instance. Tue May  1 16:02:23 2018
Run 6614643 instance 1 released for submission. Tue May  1 16:02:43 2018
(945676.0) Job Submitted at OSGFactory Tue May  1 16:02:50 2018
(945676.0) Job Done at OSGFactory Tue May  1 16:03:58 2018


Here is an example with a parameter sweep over two variables.  Parameter sweep results are returned in a directory named *runName*.  Each job in the sweep has a separate subdirectory.

In [22]:
submitCommand = SubmitCommand()
submitCommand.setWallTime(5)
submitCommand.setParameters(['@@Vin=1:2:10','@@C=100e-6,100e-5'])
submitCommand.setCommand(applicationCode)
submitCommand.setCommandArguments(['--C','@@C','--Vin','@@Vin'])
submitCommand.show()
result = submitCommand.submit()

runResultsDir = result['runName']
!ls -R $runResultsDir

submit --wallTime 5 --parameters @@Vin=1:2:10 --parameters @@C=100e-6,100e-5
       /apps/pegtut/current/examples/capacitor_voltage/sim1.py --C @@C --Vin
       @@Vin
Run 6614647 registered 1 job instance. Tue May  1 16:04:31 2018
Run 6614647 instance 0 released for submission. Tue May  1 16:04:44 2018
(945677.0) DAG Submitted at WF-OSGFactory Tue May  1 16:05:00 2018
(945677.0) DAG Running at WF-OSGFactory Tue May  1 16:05:08 2018
(945677.0) DAG Done at WF-OSGFactory Tue May  1 16:06:57 2018
Simulations complete. Results are stored in directory /home/nanohub/mmh/data/sessions/1301182/run_5Wz8uf/06614647
06614647:
01  05	09			   pegasusjobstats.csv	    pegasussummary.csv
02  06	10			   pegasusjobstats.txt
03  07	parameterCombinations.csv  pegasusstatus.txt
04  08	pegasus.analysis	   pegasussummary-time.csv

06614647/01:

06614647/02:

06614647/03:

06614647/04:

06614647/05:

06614647/06:

06614647/07:

06614647/08:

06614647/09:

06614647/10:


Alternative progress reporting methods are provided to facilitate advanced reporting. As an example the *submit* progress reporting method output is injested by Rappture and rendered as a progress bar.

In [23]:
submitCommand = SubmitCommand()
submitCommand.setWallTime(5)
submitCommand.setParameters(['@@Vin=1:2:10','@@C=100e-6,100e-5'])
submitCommand.setProgress(detail='submit')
submitCommand.setCommand(applicationCode)
submitCommand.setCommandArguments(['--C','@@C','--Vin','@@Vin'])
submitCommand.show()
result = submitCommand.submit()

submit --wallTime 5 --parameters @@Vin=1:2:10 --parameters @@C=100e-6,100e-5
       --progress submit /apps/pegtut/current/examples/capacitor_voltage/sim1.py
       --C @@C --Vin @@Vin
=SUBMIT-PROGRESS=> aborted=0 finished=0 failed=0 executing=0 waiting=10 setting_up=0 setup=0 %done=0.00 timestamp=1525205280.9
=SUBMIT-PROGRESS=> aborted=0 finished=10 failed=0 executing=0 waiting=0 setting_up=0 setup=0 %done=100.00 timestamp=1525205396.5
Simulations complete. Results are stored in directory /home/nanohub/mmh/data/sessions/1301182/run_LlRw5S/06614656


Parameter sweep variable values can also be read from a file.

In [24]:
with open("input.csv",'w') as fpInput:
   fpInput.write("@@Vin,@@C\n")
   for v in range(1,6):
      for c in (0.0001,0.001):
         fpInput.write("%f,%f\n" % (v,c))
!cat input.csv

@@Vin,@@C
1.000000,0.000100
1.000000,0.001000
2.000000,0.000100
2.000000,0.001000
3.000000,0.000100
3.000000,0.001000
4.000000,0.000100
4.000000,0.001000
5.000000,0.000100
5.000000,0.001000


In [25]:
submitCommand = SubmitCommand()
submitCommand.setWallTime(5)
submitCommand.setDataFile('input.csv')
submitCommand.setCommand(applicationCode)
submitCommand.setCommandArguments(['--C','@@C','--Vin','@@Vin'])
submitCommand.show()
result = submitCommand.submit()

submit --wallTime 5 --data input.csv
       /apps/pegtut/current/examples/capacitor_voltage/sim1.py --C @@C --Vin
       @@Vin
Run 6614664 registered 1 job instance. Tue May  1 16:10:36 2018
Run 6614664 instance 0 released for submission. Tue May  1 16:10:47 2018
(945691.0) DAG Submitted at WF-OSGFactory-RHEL6 Tue May  1 16:10:59 2018
(945691.0) DAG Running at WF-OSGFactory-RHEL6 Tue May  1 16:11:09 2018
(945691.0) DAG Done at WF-OSGFactory-RHEL6 Tue May  1 16:12:56 2018
Simulations complete. Results are stored in directory /home/nanohub/mmh/data/sessions/1301182/run_h4eIUI/06614664


The mapping of variable combinations to job number is contained in the parameterCombinations.csv file.

In [26]:
import os.path
combinationsFile = os.path.join(result['runName'],'parameterCombinations.csv')
!cat $combinationsFile